In [15]:
import pandas as pd
from urllib.request import urlretrieve
import os

In [20]:
urls = ['https://www.abs.gov.au/statistics/people/population/regional-population-age-and-sex/2021/32350DS0001_2021.xlsx', \
       'https://www.abs.gov.au/statistics/people/population/deaths-australia/2020/33020DO005_2020.xlsx', \
       'https://www.abs.gov.au/statistics/people/education/schools/2021/Table%2035b%20Counts%20of%20all%20Schools%2C%202010-2021.xlsx']

names = ['age_sex', 'pop_deaths', 'education']

In [17]:
# Creates raw_data folder
output_relative_dir = '../raw_data/'
if not os.path.exists(output_relative_dir):
    os.makedirs(output_relative_dir)
    
# Creates abs_data folder within raw_data
if not os.path.exists(output_relative_dir + 'abs_data'):
    os.makedirs(output_relative_dir + 'abs_data')

In [19]:
abs_output_dir = output_relative_dir + 'abs_data'

In [21]:
i = 0
for url in urls:
    output_dir = f"{abs_output_dir}/{names[i]}.xlsx"
    urlretrieve(url, output_dir)
    i += 1

# Manual steps required:

- Open the age_sex and pop_deaths .xlsx files
- 

## Reading in pop_deaths -

In [30]:
sheets = [1]
''', 2, 3, 4, 5, 6, 7'''
df = pd.read_excel('../raw_data/abs_data/pop_deaths.xlsx', sheet_name = sheets)